<h1 align=center>Part 1- Segment & Cluster Neighborhoods in Toronto</h1>

<h4>Importing the necessary packages and libraries</h4>

In [97]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

In [98]:
List_url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
source = requests.get(List_url).text

In [99]:
soup = BeautifulSoup(source, 'xml')

In [100]:
table=soup.find('table')

<h4>Dataframe with three columns: PostalCode, Borough, and Neighborhood</h4>

In [78]:
#dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
column_names = ['Postalcode','Borough','Neighborhood']
df = pd.DataFrame(columns = column_names)

In [80]:
# Search all the postcode, borough, neighborhood 
for tr_cell in table.find_all('tr'):
    row_data=[]
    for td_cell in tr_cell.find_all('td'):
        row_data.append(td_cell.text.strip())
    if len(row_data)==3:
        df.loc[len(df)] = row_data

In [81]:
df.head()

,Postalcode,Borough,Neighborhood
0,M1A,Not assigned,
1,M2A,Not assigned,
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront


<h4>Data Cleaning</h4>
<h5>Remove rows where Borough is 'Not assigned' and Reassigning Neighborhood to Borough column if Neighborhood is 'Not Assigned'</h5>

In [83]:
df=df[df['Borough']!='Not assigned']

In [82]:
df['Neighborhood'].replace(0,'Not assigned', inplace = True)

In [ ]:
df[df['Neighborhood']=='Not assigned'] = df['Borough']

In [90]:
df.head()

,Postalcode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront
5,M6A,North York,Lawrence Manor / Lawrence Heights
6,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government


<h4>More than one neighborhood can exist in one postal code area</h4>

In [84]:
temp_df=df.groupby('Postalcode')['Neighborhood'].apply(lambda x: "%s" % ', '.join(x))
temp_df=temp_df.reset_index(drop=False)
temp_df.rename(columns={'Neighborhood':'Neighborhood_joined'},inplace=True)

In [91]:
df_merge = pd.merge(df, temp_df, on='Postalcode')

In [92]:
df_merge.drop(['Neighborhood'],axis=1,inplace=True)

In [93]:
df_merge.drop_duplicates(inplace=True)

In [94]:
df_merge.rename(columns={'Neighborhood_joined':'Neighborhood'},inplace=True)

In [95]:
df_merge.head()

,Postalcode,Borough,Neighborhood
0,M3A,North York,"Parkwoods, Parkwoods"
2,M4A,North York,"Victoria Village, Victoria Village"
4,M5A,Downtown Toronto,"Regent Park / Harbourfront, Regent Park / Harb..."
6,M6A,North York,"Lawrence Manor / Lawrence Heights, Lawrence Ma..."
8,M7A,Downtown Toronto,"Queen's Park / Ontario Provincial Government, ..."


<h4>.Shape method to print the dataframe </h4>

In [96]:
df_merge.shape

(103, 3)